# About
Original idea from "Have a donut", by Andy Sloane.  
Visit https://www.a1k0n.net/2011/07/20/donut-math.html for more.  
transformed from C to Python.

### Start - Making a Donut!
  
The basic idea is render a 3D object onto a 2D screen. Then create a "illumination" on the surface of each pixel point of the object. 

To start, we first need to understand the creation of a donut (torus). The simple idea is creating a 2D circle with some distance away from an axis. Then we perform so called [solid of revolution](https://en.wikipedia.org/wiki/Solid_of_revolution). Using 2D shape and rotate around the axis and rotate from $0$ to $2\pi$.  
<img src="./assets/images/Soild_of_rev.png" style="display: block; margin: 0 auto" />
  
The equation to draw a circle with R2 radius from the center axis and a circle of R1 radious is as follow:  
  
$$(x, y, z) = (R_2, 0, 0) + (R_1 cos(\theta), R_1 sin(\theta), 0)$$  
  
$$=(R_2 + R_1 cos(\theta), R_1 sin(\theta), 0)$$


In [ ]:
# Using Turtle to visualize (x, y, z) calculation.
import turtle
import numpy as np
###
R1 = 20
R2 = 50
###
main = turtle.getscreen()
t1 = turtle.Turtle()
t2 = turtle.Turtle()
t2.up()
t1.goto(R2, 0)
t1.goto(R2, R1)
t1.up()
t1.goto(R1+R2, 0)
t1.down()
t2.goto(0, 2*R2)
t2.down()
t2.goto(0, -2*R2)

### revolution torus position array = (x, y, z) ###
for THETA in np.arange(0, 2*np.pi + 0.05, 0.05):
    position_array = np.array([R2, 0, 0]) + np.array([R1 * np.cos(THETA), R1 * np.sin(THETA), 0])
    t1.goto(position_array[0], position_array[1])

t1.hideturtle()
t2.hideturtle()

turtle.done()

Then we need to perform [solid of revolution](https://en.wikipedia.org/wiki/Solid_of_revolution) of created circle. The core of solid of revolution is to rotate the circle around the y-axis by another angle $\phi$. The rotation axis in here would be the same as the imaginary axis when createing the torus with distance R2 away from center of the 2D circle.  
To do this we need the dot product of the position array with the [Rotation matrix](https://en.wikipedia.org/wiki/Rotation_matrix).  
Rotation matrix of y axis is defined as  
\begin{align*}
R_y(\phi) = 
\begin{bmatrix}
cos\phi & 0 & sin\phi \\
0 & 1 & 0 \\
-sin\phi & 0 & cos\phi
\end{bmatrix}
\end{align*}  

Which our new position array would be:  
$$position array_{NEW} = position array_{OLD} \cdot R_y(\phi)$$

In [ ]:
import turtle
import numpy as np

###
R1 = 20
R2 = 50
###

main = turtle.getscreen()
t1 = turtle.Turtle()

for THETA in np.arange(0, 2*np.pi + 0.05, 0.05):
    position_array = np.array([R2, 0, 0]) + np.array([R1 * np.cos(THETA), R1 * np.sin(THETA), 0])
    for PHI in np.arange(0, 2*np.pi + 0.05, 0.05):
        position_array = position_array.dot(np.array([[np.cos(PHI), 0, np.sin(PHI)],[0, 1 ,0], [-np.sin(PHI), 0, np.cos(PHI)]]))
        print(position_array)
        t1.goto(position_array[0], position_array[1])

t1.hideturtle()
turtle.done()

Almost looks like 3D printing! If you followed though the trutle, you will see how the process is actually being calculated, which is very cool! The result will take some time to print, but as all good things in life, it's worth the wait.
###### It should looks something like this:
<img src="./assets/images/Donut.png" style="display: block; margin: 0 auto" />  

### Spin the Donut - A lot of Math:  
Now we have our amazing donut in 3D, the viewer cannot perceive them as it is! so to take it further, let's combine additonal position matrices that rotate the torus on x-axis and z-axis as well.  
\begin{align*}
R_x(X) = 
\begin{bmatrix}
1 & 0 & 0 \\
0 & cosX & -sinX \\
0 & sinX & cosX
\end{bmatrix}
R_z(Z) = 
\begin{bmatrix}
cosZ & -sinZ & 0 \\
sinZ & cosZ & 0 \\
0 & 0 & 1
\end{bmatrix}
\end{align*}  
The dot product of the above two matrices should be something like this:  
\begin{align*}
R = R_x(X)R_z(Z) = 
\begin{bmatrix}
cosZ & -sinZ & 0 \\
cosX*sinZ & cosX*cosZ & -sinX \\
sinX*sinZ & sinX*cosZ & cosX
\end{bmatrix}
\end{align*}

This should save a lot of time for computation and solve the position matrix much quicker than letting the computer does the dot product! Finally our position matrix would be the dot product of the $(x, y, z) = (R_2 + cos\theta, R_1 sin\theta, 0)$ with $combined\ rotation\ matrix = R(X, \theta, Z)$
  
$$Rotation\ Matix (\phi, X, Z):$$  

\begin{align*}

\begin{bmatrix}
cos\phi*cosZ + sin\phi*sinX*sinZ & -cos\phi*sinZ+sin\phi*sinX*cosZ & sin\phi*cosX \\
cosX*sinZ & cosX*cosZ & -sinX \\
-sin\phi*cosZ + cos\phi*sinX*sinZ & sin\phi*sinZ + cos\phi*sinX*cosZ & cos\phi*cosX
\end{bmatrix} \\
\end{align*}  


\begin{align*}
Combined\ Rotation\ Matix (\theta, \phi, X, Z)=
\begin{bmatrix}
(R_2 + R_1 cos\theta)(cos\phi*cosZ + sin\phi*sinX*sinZ) + (R_1 sin\theta)(cosX*sinZ) + (0)(-sin\phi*cosZ + cos\phi*sinX*sinZ) \\
(R_2 + R_1 cos\theta)(-cos\phi*sinZ+sin\phi*sinX*cosZ) + (R_1 sin\theta)(cosX*cosZ) + (0)(sin\phi*sinZ + cos\phi*sinX*cosZ) \\
(R_2 + R_1 cos\theta)(sin\phi*cosX) + (R_1 sin\theta)(-sinX) + (0)(cos\phi*cosX)
\end{bmatrix}
\end{align*}

It's so long you can't even see it all in markdown! Fortunately we can simplifiy everything!
The simplified complete Rotation Matrix ($\phi$, X, Z) should be as follow:  

\begin{align*}
\begin{bmatrix}
x \\ y \\ z  
\end{bmatrix}
=
\begin{bmatrix}
(R_2 + R_1cos\theta)(cosZcos\phi + sinXsinZsin\phi) - R_1cosXsinZsin\theta \\
(R_2 + R_1cos\theta)(cos\phi sinZ - cosZsinXsin\phi) - R_1cosXsinZsin\theta \\
cosX(R_2 + R_1cos\theta)sin\phi + R_1sinXsin\theta
\end{bmatrix}
\end{align*}
  
Now let's calculate them!

In [ ]:
import numpy as np
import turtle

###
X = 10
Z = 5
R_1 = 20
R_2 = 50
###

main = turtle.getscreen()
t1 = turtle.Turtle()

cos_X = np.cos(X)
sin_X = np.sin(X)
cos_Z = np.cos(Z)
sin_Z = np.sin(Z)

for THETA in np.arange(0, 2*np.pi + 0.05, 0.05):
    cos_THETA = np.cos(THETA)
    sin_THETA = np.sin(THETA)
    for PHI in np.arange(0, 2*np.pi + 0.05, 0.05):
        cos_PHI = np.cos(PHI)
        sin_PHI = np.sin(PHI)
        position_array = [((R_2 + R_1 * cos_THETA) * (cos_Z * cos_PHI + sin_X * sin_Z * sin_PHI) - R_1 * cos_X * sin_Z * sin_THETA),
                          ((R_2 + R_1 * cos_THETA) * (cos_PHI * sin_Z - cos_Z * sin_X * sin_PHI) + R_1 * cos_X * sin_Z * sin_THETA),
                          (cos_X * (R_2 + R_1 * cos_THETA) * sin_PHI + R_1 * sin_X * sin_THETA)
                          ]
        t1.goto(position_array[0], position_array[1])

t1.hideturtle()
turtle.done()

<img src="./assets/images/3DDonut.png" style="display: block; margin: 0 auto" />
Oh wow! We are so close! This well positioned, well calculated, and well balanced perfection of a donut. Now we have 3D coordinate for all donut, we just need to translate the 3D coordinate and project it onto a 2D plane! 

### Donut projection - Render the Donut:
Now we have a donut position array (x, y, z) we need to find a way to project those coordinates correctly onto the 2D plane. To start, let's calculate the rendering algorithm that translate 3D coordinate.  
<img src="./assets/images/Rendering.png" style="display: block; margin: 0 auto" />
By projecting each point (x, y, z) onto plane distance z' from eye. So our algorithm should translate the 3D coordinates (x, y, z) onto the corresponding 2D coordinates (x', y', z') where z' is a constant. Very much like the 2D images of the donut that's drawn by turtle before! But if you fiddle around the X and Z values, sometimes the projection images looks not like a donut at all! This is why we are translate the cooredinate propperly so we can displace them onto terminal.

As we can see from the image, the donut coordinates (x, y, z) would be proportional to projection coordinates (x', y', z') and they all form a right trangle. So their revlative proportion must also be maintained.
$$ \frac{y'}{z'} = \frac{y}{z} $$
$$ y' = \frac{yz'}{z} $$  
Similarly for x coordinate:
$$ x' = \frac{xz'}{z} $$  
In conclusion, 2D coordinates for our torus should be something like this:  
$$ (x', y') = (\frac{xz'}{z}, \frac{yz'}{z}) $$  
Let's put it to test!!

In [28]:
import numpy as np
import turtle

###
X = 1
Z = 2
R_1 = 1
R_2 = 3
Z_render = 5
Z_PROJECTION = 100* Z_render*3/(8*(R_1+R_2))
###

main = turtle.getscreen()
t1 = turtle.Turtle()

cos_X = np.cos(X)
sin_X = np.sin(X)
cos_Z = np.cos(Z)
sin_Z = np.sin(Z)


for THETA in np.arange(0, 2*np.pi + 0.05, 0.05):
    cos_THETA = np.cos(THETA)
    sin_THETA = np.sin(THETA)
    for PHI in np.arange(0, 2*np.pi + 0.05, 0.05):
        cos_PHI = np.cos(PHI)
        sin_PHI = np.sin(PHI)
        circle_x = R_2 + R_1 * cos_THETA
        circle_y = R_1 * sin_THETA
        x = circle_x*(cos_Z*cos_PHI + sin_X*sin_Z*sin_PHI)- circle_y*cos_X*sin_Z
        y = circle_x*(sin_Z*cos_PHI - sin_X*cos_Z*sin_PHI)+ circle_y*cos_X*cos_Z
        z = Z_render + cos_X*circle_x*sin_PHI + circle_y*sin_X
        ooz = 1/z
        x_2d = (Z_PROJECTION * ooz * x)
        y_2d = (Z_PROJECTION * ooz * y)
        # print(x_2d, y_2d)
        t1.goto(x_2d, y_2d)

t1.hideturtle()
turtle.done()

Wow, Looks like nothing changed. However, with the inclusion of the Z coordinate, the torus now can be seen at a actual 2D representation of our 3D torus! Which before, we ignored the thrid dimension and while it looks 3D like, the closer the donut is the size of the donut stayed the same. While our new awsome projection contain a proper real life view of the donut.  
Now that's done, all we need is to calculate the surface normal and render the illumination of our torus.

### Surface Normal - Shine a light:  
Surfce normal seems difficult to calculate, but in actuality it's the dot product of the normal from our calcuated position (x, y, z)with our light source. which we have pretty much all calculated! Only difference is that instead finding the exact position the starting point will only be $(cos\theta, sin\theta, 0)$  
$$ (N_x, N_y, N_z) = (cos\theta, sin\theta, 0) \cdot Rotation\ Matrix(\phi, X, Z)$$  

\begin{align*}
(N_x, N_y, N_z) = 
\begin{bmatrix}
(R_2 + R_1 cos\theta)(cos\phi*cosZ + sin\phi*sinX*sinZ) + (R_1 sin\theta)(cosX*sinZ) + (0)(-sin\phi*cosZ + cos\phi*sinX*sinZ) &
(R_2 + R_1 cos\theta)(-cos\phi*sinZ+sin\phi*sinX*cosZ) + (R_1 sin\theta)(cosX*cosZ) + (0)(sin\phi*sinZ + cos\phi*sinX*cosZ) &
(R_2 + R_1 cos\theta)(sin\phi*cosX) + (R_1 sin\theta)(-sinX) + (0)(cos\phi*cosX)
\end{bmatrix}
\end{align*}

The short version for normals $(N_x, N_y, N_z)$ is following:
$$ [{cosX sin\theta sinZ+cos\theta (cos\phi cosZ+sin\phi sinX sinZ),\ cosX cosZ sin\theta +cos\theta (cosZ sin\phi sinX-cos\phi sinZ),\ cos\theta cosX sin\phi -sin\theta sinX}] $$

Now we have our surface normal. We can perform dot product of our surface normal with our light, which is somwhere normalized at (0, 1, -1).
$$ L = (N_x, N_y, N_z) \cdot \begin{bmatrix}0 \\ 1 \\ -1\end{bmatrix} $$
$$ L = cos\phi cos\theta sinZ - cosX cos\theta sin\phi -sinX sin\theta + cosZ*(cosXsin\theta - cos\theta sinX sin\phi) $$

### Illumination - Shade the Donut!
Now we have three main component:
1. Torus coordinates: **(x, y, z)**
2. Projection coordinates: **(x, y)**
3. Surface Normal: **L**  
  
Time to build our 3D donut! Before we build our donut with ASCII characters as level of illumination, we can use pen thickness to do the same. Which works perfectlly with turtle. I've also excluded the normalized point that's negative (normal points away from the view). It helps with computation and the speed of the drawing. Of course, when we printing them onto console with ASCII, I would be using technique such as Z-Buffer to actually check the point, but for simple drawing, lazy one such as mine works just fine!

Now by limit the level of illumination to 2 we can see a clear features of the light on the donut:  

In [9]:
import numpy as np
import turtle

###
X = 1
Z = 2
R_1 = 3
R_2 = 5
Z_render = 10
Z_PROJECTION = 100* Z_render*3/(8*(R_1+R_2))
###

main = turtle.getscreen()
t1 = turtle.Turtle()

cos_X = np.cos(X)
sin_X = np.sin(X)
cos_Z = np.cos(Z)
sin_Z = np.sin(Z)

for THETA in np.arange(0, 2*np.pi + 0.05, 0.05):
    cos_THETA = np.cos(THETA)
    sin_THETA = np.sin(THETA)
    for PHI in np.arange(0, 2*np.pi + 0.05, 0.05):
        cos_PHI = np.cos(PHI)
        sin_PHI = np.sin(PHI)
        circle_x = R_2 + R_1 * cos_THETA
        circle_y = R_1 * sin_THETA
        x = circle_x*(cos_Z*cos_PHI + sin_X*sin_Z*sin_PHI)- circle_y*cos_X*sin_Z
        y = circle_x*(sin_Z*cos_PHI - sin_X*cos_Z*sin_PHI)+ circle_y*cos_X*cos_Z
        z = Z_render + cos_X*circle_x*sin_PHI + circle_y*sin_X
        ooz = 1/z
        x_2d = (100/2 + Z_PROJECTION*ooz*x)
        y_2d = (100/2 - Z_PROJECTION*ooz*y)
        
        L = cos_PHI * cos_THETA * sin_Z - cos_X * cos_THETA * sin_PHI - sin_X * sin_THETA + cos_Z * (cos_X * sin_THETA - cos_THETA * sin_X * sin_PHI)
        if L < 0:
            break
        t1.width(int(L*2))
        t1.goto(x_2d, y_2d)

t1.hideturtle()
turtle.done()

#### Perfect! ####  
<img src="./assets/images/SurfaceNormal.png" style="display: block; margin: 0 auto" />  
Now we only need to find a way to print ASCII character onto a console of a sort. Of course, feel free to play around the levels of illumination, some may not work very well if the size of the donut is small. We have all the pieces of code that we need to make a ASCII donut!


### Pygame - Console ASCII maker ###
Before we dive into the donut making process with pygame, we first need to figure out how to display a pygame window that will allow ASCII character to be displayed. Of course this is not about pygame "how to", so I will make something simple. For all we need, we need a character display with a set window size, 500 x 500 should be more then enough.

In [13]:
import numpy as np

def donutInstance(X: int, Z: int):
    output = np.chararray((500, 500))
    print(output)
    R_1 = 1
    R_2 = 2
    Z_render = 5
    Z_PROJECTION = 500* Z_render*3/(8*(R_1+R_2))

    cos_X = np.cos(X)
    sin_X = np.sin(X)
    cos_Z = np.cos(Z)
    sin_Z = np.sin(Z)
    for THETA in np.arange(0, 2*np.pi + 0.05, 0.05):
        cos_THETA = np.cos(THETA)
        sin_THETA = np.sin(THETA)
        for PHI in np.arange(0, 2*np.pi + 0.05, 0.05):
            cos_PHI = np.cos(PHI)
            sin_PHI = np.sin(PHI)
            circle_x = R_2 + R_1 * cos_THETA
            circle_y = R_1 * sin_THETA
            x = circle_x*(cos_Z*cos_PHI + sin_X*sin_Z*sin_PHI)- circle_y*cos_X*sin_Z
            y = circle_x*(sin_Z*cos_PHI - sin_X*cos_Z*sin_PHI)+ circle_y*cos_X*cos_Z
            z = Z_render + cos_X*circle_x*sin_PHI + circle_y*sin_X
            ooz = 1/z
            x_2d = int(500/2 + Z_PROJECTION*ooz*x)
            y_2d = int(500/2 - Z_PROJECTION*ooz*y)
            
            L = cos_PHI * cos_THETA * sin_Z - cos_X * cos_THETA * sin_PHI - sin_X * sin_THETA + cos_Z * (cos_X * sin_THETA - cos_THETA * sin_X * sin_PHI)
            if L < 0:
                break
            luminance_index = int(L*8)
            point_char = ".,-~:;=!*#$@"[luminance_index]
            output[x_2d][y_2d] = point_char
    
    return output

In [2]:
print(donutInstance(1, 3))

[[b'P' b'\x01' b'\xf1' ... '' '' '']
 ['' '' '' ... '' '' '']
 [b'|' '' '' ... '' '' '']
 ...
 [b'\xa6' b'\x03' '' ... b'\x02' '' '']
 ['' '' '' ... b'\xe0' b'\x02' '']
 [b'e' b'\xc6' b'd' ... '' '' '']]
[[b'P' b'\x01' b'\xf1' ... '' '' '']
 ['' '' '' ... '' '' '']
 [b'|' '' '' ... '' '' '']
 ...
 [b'\xa6' b'\x03' '' ... b'\x02' '' '']
 ['' '' '' ... b'\xe0' b'\x02' '']
 [b'e' b'\xc6' b'd' ... '' '' '']]


In [14]:
import pygame
import numpy

pygame.init()

BLACK = (0, 0, 0)

display_surface = pygame.display.set_mode((500, 500))
font = pygame.font.Font('freesansbold.ttf', 32)
donut = donutInstance(1,3)
for pixlex in range(500):
    for pixley in range(500):
        digit = donut[pixley][pixlex]
        ASCII_digit = font.render(digit, True, (250, 250, 250))
        textRect = ASCII_digit.get_rect()
        textRect.center = (pixley, pixlex)
        display_surface.blit(ASCII_digit, textRect)

try:
    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
        pygame.display.update()
except pygame.error:
    print("Exit complete.")




[['' b'\xb0' b'\x8f' ... '' b'H' '']
 [b'D' '' b'A' ... '' b'}' '']
 [b'\\' '' b'R' ... '' b'd' '']
 ...
 ['' '' '' ... '' '' '']
 ['' '' '' ... '' '' '']
 ['' '' '' ... '' '' '']]
Exit complete.


That could be better! But hey if it works then our logic is correct! It only needs extra fixes to making it looks more pleasing than that mess. To continue, I'll alter the code significantly 

In [4]:
import pygame
import numpy as np

pygame.init()

white = (255, 255, 255)
black = (0, 0, 0)
hue = 0

WIDTH = 1920
HEIGHT = 1080

x_start, y_start = 0, 0

R_2 = 10
R_1 = 20

rows = HEIGHT // R_2
columns = WIDTH // R_1
screen_size = rows * columns

Z_render = 5
Z_PROJECTION = 500* Z_render*3/(8*(R_1+R_2))

x_offset = columns / 2
y_offset = rows / 2

X, Z = 0, 0  # rotating animation

theta_spacing = 10
phi_spacing = 1 # for faster rotation change to 2, 3 or more, but first change 86, 87 lines as commented

chars = ".,-~:;=!*#$@"  # luminance index

screen = pygame.display.set_mode((WIDTH, HEIGHT))

display_surface = pygame.display.set_mode((WIDTH, HEIGHT))
# display_surface = pygame.display.set_mode((0, 0), pygame.FULLSCREEN)
pygame.display.set_caption('Donut')
font = pygame.font.SysFont('Arial', 18, bold=True)



def text_display(letter, x_start, y_start):
    text = font.render(str(letter), True, white)
    display_surface.blit(text, (x_start, y_start))

# def text_display(letter, x_start, y_start):
#     text = font.render(str(letter), True, white)
#     display_surface.blit(text, (x_start, y_start))

while True:

    screen.fill((black))

    z = [0] * screen_size  # Donut. Fills donut space
    b = [' '] * screen_size  # Background. Fills empty space

    cos_X = np.cos(X)
    sin_X = np.sin(X)
    cos_Z = np.cos(Z)
    sin_Z = np.sin(Z)
    for THETA in np.arange(0, 2*np.pi + 0.05, 0.05):
        cos_THETA = np.cos(THETA)
        sin_THETA = np.sin(THETA)
        for PHI in np.arange(0, 2*np.pi + 0.05, 0.05):
            cos_PHI = np.cos(PHI)
            sin_PHI = np.sin(PHI)
            circle_x = R_2 + R_1 * cos_THETA
            circle_y = R_1 * sin_THETA
            x = circle_x*(cos_Z*cos_PHI + sin_X*sin_Z*sin_PHI)- circle_y*cos_X*sin_Z
            y = circle_x*(sin_Z*cos_PHI - sin_X*cos_Z*sin_PHI)+ circle_y*cos_X*cos_Z
            z = Z_render + cos_X*circle_x*sin_PHI + circle_y*sin_X
            ooz = 1/z
            x_2d = int(500/2 + Z_PROJECTION*ooz*x)
            y_2d = int(500/2 - Z_PROJECTION*ooz*y)
            o = x_2d + y_2d
            L = cos_PHI * cos_THETA * sin_Z - cos_X * cos_THETA * sin_PHI - sin_X * sin_THETA + cos_Z * (cos_X * sin_THETA - cos_THETA * sin_X * sin_PHI)
            luminance_index = int(L*8)

            if L > 0 and ooz > z[o]:
                z[o] = ooz
                b[o] = chars[luminance_index if luminance_index > 0 else 0]

    if y_start == rows * R_1 - R_1:
        y_start = 0

    for i in range(len(b)):
        X += 0.00004  # for faster rotation change to bigger value
        Z += 0.00002  # for faster rotation change to bigger value
        if i == 0 or i % columns:
            text_display(b[i], x_start, y_start)
            x_start += R_2
        else:
            y_start += R_1
            x_start = 0
            text_display(b[i], x_start, y_start)
            x_start += R_2


    pygame.display.update()

    hue += 0.005

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                pygame.quit()

IndexError: invalid index to scalar variable.